In [1]:
import time
start = time.time()
from selenium import webdriver
#from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
options = webdriver.ChromeOptions()
options.add_argument('headless')
#user-agent
options.add_argument('user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.80 Safari/537.36')


import requests
from bs4 import BeautifulSoup
import pandas as pd

import openpyxl#엑셀저장


URL = 'https://www.saramin.co.kr/zf_user/jobs/list/job-category?cat_mcls=2&sort=RD'
I_URL = 'https://www.saramin.co.kr/zf_user/jobs/relay/view?'
LIMIT = 1

In [2]:
def extract_info(html):
    global LIMIT
    #inx
    idx = html['id'][4:]
   # stacks = extract_stack(idx)
        #pages
    
    
    
    #회사명
    company = html.find("div",{"class":"company_nm"}).find("a").string
    #제목
    title = html.find("div",{"class":"job_tit"}).find("a").string
    #경력
    career = html.find("p",{"class":"career"}).string
    #학력
    education = html.find("p",{"class":"education"}).string
    #근무형태
    employment_t = html.find("p",{"class":"employment_type"}).string
    #근무장소
    try:
        place = html.find("p",{"class":"work_place"}).string
    except Exception as e:
        place = "None"
    #마감일
    deadline_tmp = html.find("p",{"class": "deadlines"}).get_text()
    deadlines = deadline_tmp[:deadline_tmp.find('(')]

    #등록일
    reg = html.find("span",{"class":"reg_date"}).string
    
    all_infos = {'title':title, 'company':company, 'career':career, 'education':education,
                'employment_type' :employment_t,  'workc_place':place, 'deadline': deadlines,
                'idx':idx,'reg':reg}
    if(reg == '(1일 전 등록)'):
        LIMIT = 0
        return all_infos
    return all_infos

In [3]:
def extract_page(pages):#전체페이지
    result = requests.get(f"{URL}&page={pages}")

    soup = BeautifulSoup(result.text, "html.parser")
    #모든 공고 리스트
    infos= soup.find_all("div", {"class":"list_item"})
    
    for_test = 1
    jobs = []
    for info in infos:# 각 공고 접근
        job = extract_info(info)
        jobs.append(job)
        if(LIMIT == 0):# 1일전 등록 check
            return jobs
        #test
        '''
        if(for_test > 10):
            return jobs
        for_test = for_test + 1
        '''
        #test

    return jobs

In [4]:
def get_stack(a_jobs):#세부페이지 태그
    #페이지 열기
    browser = webdriver.Chrome("/usr/local/bin/chromedriver",chrome_options=options);
    browser.implicitly_wait(2)
    
    for e_info in a_jobs:
        d_url = 'https://www.saramin.co.kr/zf_user/jobs/relay/view?rec_idx='+e_info['idx']
        #페이지 접속
        browser.get(d_url)
        #오류제거
        wait = WebDriverWait(browser,20)
        element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,'.tags')))
        #tag접근
        tag_sec = browser.find_element(By.CSS_SELECTOR,'.tags')
        stacks = tag_sec.find_element(By.CSS_SELECTOR,'.scroll')
        tmps = stacks.find_elements(By.TAG_NAME, 'a')
        #모든 태크 모으기
        stack = []
        for tmp in tmps:
            url = tmp.get_attribute('href')
            index = url.find('kewd')
            if(index != -1):
                stack.append(url[index+5:])
        e_info["stack"] = stack

    browser.quit()
    return a_jobs

In [5]:
def main():
    global LIMIT
    a_jobs = []
    i = 1
    while(LIMIT):
        a_jobs.extend(extract_page(i))
        i = i+1
    
    result = get_stack(a_jobs)
    
        
    return result

In [6]:
data = main()

/var/folders/8l/z4qr6xns7sb_mbw_25xwx0cm0000gn/T/ipykernel_45540/1707580780.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome("/usr/local/bin/chromedriver",chrome_options=options);
/var/folders/8l/z4qr6xns7sb_mbw_25xwx0cm0000gn/T/ipykernel_45540/1707580780.py:3: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome("/usr/local/bin/chromedriver",chrome_options=options);


In [7]:
dataA = pd.DataFrame(data)

In [9]:
dataA.to_excel("220210.xlsx")
end = time.time()
print(format(end-start))

2264.5981340408325


In [11]:
dataA.to_excel("220210_del_index.xlsx",index=False)